# Features Constantes y Quasi-Constantes

En el presente cuaderno se explora cómo manejar features constantes y quasi-constantes en un data set. Dicho manejo se realiza mediante el uso de herramientas de Scikit-learn, así como manualmente.

---
## Features constantes

Estos son aquellos features cuyo valor no cambia. Puede visualizarse como una columna que tiene siempre el mismo valor para todas sus filas. Evidentemente, un feature de este tipo no brinda información valiosa para realizar predicciones con un modelo de machine learning, razón por la cual se eliminan y no se toman en cuenta.

A continuación, se muestra cómo identificar y eliminar features constantes utilizando VarianceThreshold de Scikit-learn y utilizando código normal.

In [50]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

data = pd.read_csv('notebooks/dataset_1.csv')

### VarianceThreshold

Mostramos primero cómo trabajar con VarianceThreshold. Es importante resaltar que este método solo sirve para variables numéricas.

**Importante:** el estudio para la selección de variables debería realizarse solo sobre el conjunto de train, pero para efectos de este resumen, se utilizará todo el dataset.

In [43]:
# se seleccionan solo las variables cuya varianza sea mayor a 0
sel = VarianceThreshold(threshold=0) # especifica que varianza debe ser mayor a 0
sel.fit(data)  # busca las variables cuya varianza es 0
print(sel.get_support()) # muestra cuáles variables se tomarán en cuenta (True)
print(sum(sel.get_support())) # muestra cuántas variables se tomarán en cuenta

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True False  True
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False False  True  True  True
  True  True False  True False  True  True False  True  True  True  True
 False  True False  True  True  True  True  True  True  True  True  True
  True  True  True False False  True  True  True  True  True  True False
  True False  True  True  True  True False  True  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  T

In [44]:
# escogemos las columnas que son constantes 
constant = data.columns[~sel.get_support()] # nótese el uso de ~ para indicar que queremos las que no son constantes
print("Variables constantes:", len(constant), "\n") # también: print(sum(~sel.get_support()))

# incluso podemos visualizar que cada una de estas columnas solo tiene un valor repetido
for col in constant:
    print(col, data[col].unique())

Variables constantes: 33 

var_23 [0]
var_33 [0]
var_44 [0]
var_61 [0]
var_80 [0]
var_81 [0]
var_87 [0]
var_89 [0.]
var_92 [0]
var_97 [0]
var_99 [0]
var_112 [0]
var_113 [0]
var_120 [0]
var_122 [0]
var_127 [0]
var_135 [0]
var_158 [0]
var_167 [0]
var_171 [0]
var_178 [0.]
var_180 [0.]
var_182 [0]
var_195 [0]
var_196 [0]
var_201 [0]
var_212 [0]
var_215 [0]
var_225 [0]
var_227 [0.]
var_248 [0]
var_294 [0]
var_297 [0]


In [45]:
# VarianceThreshold retorna un numpy array, entonces hay que guardar el nombre de las columnas 
# para poder reconstruir un DataFrame
columns = data.columns[sel.get_support()]

data = sel.transform(data) # en esta etapa, se transformaría el train y el test
data = pd.DataFrame(data, columns=columns)
data.head()

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_290,var_291,var_292,var_293,var_295,var_296,var_298,var_299,var_300,target
0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
1,0.0,0.0,0.0,3.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
2,0.0,0.0,0.0,5.88,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,67772.7216,0.0
3,0.0,0.0,0.0,14.10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
4,0.0,0.0,0.0,5.76,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0


### Método manual para variables numéricas

Al igual que con VarianceThreshold, realizaremos las operaciones sobre todo el dataset, pues lo que interesa es mostrar los métodos. No obstante, en un caso real, se debería separar en conjuntos de train y test y realizar el análisis pre-selección sobre el conjunto de train.

In [49]:
# utilizando std() de pandas, seleccionamos las columnas cuya varianza sea 0
constant_cols = [col for col in data.columns if data[col].std() == 0]

# eliminamos todas las columnas constantes
# en un caso real, el drop se aplica tanto a train como a test
data.drop(labels=constant_cols, axis=1, inplace=True)
print(len(data.columns))

268


### Método manual para variables categóricas

Si bien se podrían convertir las variables categóricas a numéricas para luego aplicar alguno de los métodos anteriores, esto implicaría tiempo de preprocesamiento que podría ser costoso en variables que de por sí no se van a utilizar.

A continuación se plantea una mejor alternativa, teniendo en cuenta la consideración sobre analizar solo los datos del conjunto de train.

In [51]:
# Puesto que el dataset que estamos estudiando es completamente numérico
# convertimos las variables a objetos para simular que son categóricas
data = data.astype('O')
print(data.dtypes.head())

# utilizando nunique() de pandas, seleccionamos las columnas cuya solo tengan 1 valor
# dado que nunique() retorna el número de valores distintos
# se puede considerar nunique(dropna=False) si hay valores nulos, pues el método los ignora por default
constant_cols = [col for col in data.columns if data[col].nunique() == 1]

# eliminamos todas las columnas constantes
# en un caso real, el drop se aplica tanto a train como a test
data.drop(labels=constant_cols, axis=1, inplace=True)
print(len(data.c))

var_1    object
var_2    object
var_3    object
var_4    object
var_5    object
dtype: object
268


---
## Features Quasi-Constantes
